# Homework

### Q1. Running Elastic

What's the version.build_hash value?

In [22]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')
info = es_client.info()

# Response
info['version']['build_hash']

'42f05b9372a9a4a470db3b52817899b99a76ee73'

### Q2. Indexing the data

Which function do you use for adding your data to elastic?

R:. index

In [23]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [24]:
index_settings={
                "settings": {
                    "number_of_shards": 1,
                    "number_of_replicas": 0
                },
                "mappings": {
                    "properties": {
                        "text": {"type": "text"},
                        "section": {"type": "text"},
                        "question": {"type": "text"},
                        "course": {"type": "keyword"} 
                        }
                    }
                }

index_name = "course-question"

# Indexing process
for doc in documents:
    es_client.index(index=index_name, document=doc)

### Q3. Searching

What's the score for the top ranking result?

- 84.50
- 64.50
- 44.50
- 24.50

R:. 44.50

In [27]:
def elastic_search(query):

    """
    """

    search_query = {
        "size": 4,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                #"filter": {
                #    "term": {
                #        "course": "data-engineering-zoomcamp"
                #    }
                #}
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    # extracting the results
    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_score'])

    return result_docs

In [28]:
elastic_search("How do execute a command on a Kubernetes pod?")

/tmp/ipykernel_165946/2205147168.py:26: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


[44.56891, 44.56891, 44.56891, 36.28301]

### Q4. Filtering

Return 3 results. What's the 3rd question returned by the search engine?

- How do I debug a docker container?
- How do I copy files from a different folder into docker container’s working      directory?
- How do Lambda container images work?
- How can I annotate a graph?


R:. How do I debug a docker container?'


In [35]:
def elastic_search(query):

    """
    """

    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    # extracting the results
    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [37]:
search_results = elastic_search("How do copy a file to a Docker container?")
search_results

/tmp/ipykernel_165946/59987896.py:26: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Launch the container im

### Q5. Building a prompt

What's the length of the resulting prompt? (use the len function)

- 946
- 1446
- 1946
- 2446


R:. 1446

In [63]:
def prompt_builder(query: str, search_results: list)->str:

    """
    """
    
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT:
    {context}
    """.strip()

    context_template = """
    Q: {question}
    A: {text}
    """.strip()

    context=""
    for doc in search_results:
        context = context+context_template.format(question=doc['question'] , text=doc['text']) #f"question: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context)

    return prompt


In [64]:
len(prompt_builder(query="How do copy a file to a Docker container?" , search_results=search_results))

1330

In [66]:
prompt = prompt_builder(query="How do copy a file to a Docker container?" , search_results=search_results)
prompt

"You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\n    Use only the facts from the CONTEXT when answering the QUESTION.\n\n    QUESTION: How do copy a file to a Docker container?\n\n    CONTEXT:\n    Q: How do I debug a docker container?\n    A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)Q: How do I debug a docker container?\n    A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)Q: How do I d

### Q6. Tokens

In [73]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [74]:
len(encoding.encode(prompt))

303

## Response

In [75]:
import os
from dotenv import load_dotenv
from openai import OpenAI

In [76]:
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [77]:
def llm(prompt:str)->str:

    """
    """
    client = OpenAI(
    api_key=OPENAI_API_KEY
    )
    
    client = OpenAI()
    response = client.chat.completions.create(
                model = "gpt-4o-mini",
                messages=[{"role":"user", "content":prompt}]
                )

    return response.choices[0].message.content

In [78]:
llm(prompt=prompt)

'The provided context does not contain information on how to copy a file to a Docker container. Therefore, I cannot answer the question based on the given context.'